In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [3]:
test_set = pd.read_csv("../data/ml-1m/testing_set_seed123_ml1m.csv",sep=",",
    header=None,
    names=["userID", "itemID", "Rating"],)
test_set


,userID,itemID,Rating
0,6039,373,4
1,6039,221,3
2,6039,2602,4
3,6039,132,4
4,6039,350,4
...,...,...,...
202331,2946,885,3
202332,2946,343,3
202333,2946,151,4
202334,2946,1792,4


In [4]:
all_users_tp = test_set.groupby('userID')['itemID'].apply(list).tolist()
all_users_tp.__len__()

6040

In [5]:


mf = "model_reco/1m/mf/reco_matrix_mf_1m_1000.npy"
wmf = "model_reco/1m/wmf/reco_matrix_wmf_1m_1000.npy"
neumf = "model_reco/1m/neumf/reco_matrix_neumf_1m_1000.npy"
vaecf = "model_reco/1m/vaecf/reco_matrix_vaecg_1m_1000.npy"
# neumf = "model_reco/1m/neumf/reco_matrix_neumf_1m_1000_sig.npy"
# neumf = "model_reco/1m/neumf/reco_matrix_neumf_1m_1000_sig__40.npy"
reco_matrix = np.load(f"../{mf}")
reco_matrix.shape

(1, 6040, 1000)

In [6]:

from cornac.metrics import RMSE, AUC, NDCG, Precision, Recall,HitRatio

hr=HitRatio(20)
alist=[]
o=0

sum_hr=0
for u in range(reco_matrix[o].shape[0]):
    x=hr.compute(all_users_tp[u],reco_matrix[o][u][:20])
    sum_hr=sum_hr+x
plain_hr=sum_hr/reco_matrix[o].shape[0]

plain_hr


/Users/tahsinalamgirkheya/anaconda3/envs/re-ranking/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


0.34735099337748343

In [7]:
mf
0.26225165562913905
0.25149006622516556
0.2640728476821192
0.30082781456953644
0.3208609271523179
0.3350993377483444
0.33741721854304635
0.34205298013245033
0.34735099337748343

0.34735099337748343

In [8]:
# sig
0.43890728476821195
0.4294701986754967
0.42450331125827817
0.42201986754966886
0.4155629139072848
0.39536423841059604
0.38625827814569536
0.3753311258278146
0.373841059602649

0.373841059602649

In [9]:
newlist[index]

NameError: name 'newlist' is not defined

In [10]:
0.34735099337748343
0.32466887417218543
0.30132450331125826
0.2850993377483444
0.26589403973509934
0.2599337748344371
0.2458609271523179
0.2367549668874172
0.226158940397351

0.226158940397351

In [11]:
# from NDCG import NDCG
ndcg=NDCG(20)
alist=[]
sum_ndcg=0
o = 0
for u in range(reco_matrix[0].shape[0]):
    x=ndcg.compute(all_users_tp[u],reco_matrix[o][u][:20])
    
    # print(f"x {x} tp {all_users_tp[u]} reco {reco_matrix[u][:20]}")
    # alist.append(x)
    sum_ndcg=sum_ndcg+x
    # print(x)
plain_ndcg=sum_ndcg/reco_matrix[o].shape[0]

plain_ndcg
# 0.06877010356676778

0.03625590827246503

In [12]:
all_pds =[]
reco_matrix_fair=[]
sens="occupation"
model = "mf"
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_1.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_2.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_3.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_4.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_5.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_6.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_7.csv",sep=",",))
all_pds.append(pd.read_csv(f"../results/1m/{model}/{sens}_8.csv",sep=",",))
# results/reco_mat_test.npy
for j in range(len(all_pds)):
    reco_m =[]
    for i in range(all_pds[j].__len__()):
        reco_m.append(np.array(all_pds[j].iloc[i][1:]))
    reco_matrix_fair.append(reco_m)
    



In [13]:
reco_matrix = reco_matrix[0]

In [14]:

hr=HitRatio(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_hr]+hrlist

[0.34735099337748343,
 0.35165562913907283,
 0.35894039735099337,
 0.36854304635761587,
 0.37317880794701985,
 0.3741721854304636,
 0.3718543046357616,
 0.37268211920529803,
 0.3746688741721854]

In [15]:


hr=NDCG(20)
hrlist=[]
for r in reco_matrix_fair:
    sum_hr=0
    for u in range(reco_matrix.shape[0]):
        x=hr.compute(all_users_tp[u],r[u])
        sum_hr=sum_hr+x
    hr_occ=sum_hr/reco_matrix.shape[0]
    hrlist.append(hr_occ)
    

[plain_ndcg]+hrlist


[0.03625590827246503,
 0.03662558029469883,
 0.03724325760086126,
 0.03773750969407651,
 0.03764982722158577,
 0.03657585667488931,
 0.03619627121786498,
 0.036224315206395956,
 0.03514872569964683]

In [16]:
movie = "../data/ml-1m/i_id_mapping_genre.csv"

df_movie = pd.read_csv(movie, sep="\t")
print(f"shape: {df_movie.shape}")


df_movie.columns = ["item_id", "Name", "genres", "itemID"]
df_movie[:4]
movies = df_movie.sort_values(by="itemID")
unique_genres = [
    "Action",
    "Thriller",
    "Romance",
    "Western",
    "Children's",
    "Mystery",
    "Fantasy",
    "Film-Noir",
    "Documentary",
    "Comedy",
    "Adventure",
    "Sci-Fi",
    "Horror",
    "Crime",
    "Musical",
    "War",
    "Animation",
    "Drama",
]
for genre in unique_genres:
    movies[genre] = 0

for index, row in movies.iterrows():
    genres = row["genres"].split("|")
    for genre in genres:
        movies.at[index, genre] = 1
movies
movies = movies.drop(columns=["item_id"])
movies = movies.sort_values(by="itemID")
movies

movies

shape: (3416, 4)


,Name,genres,itemID,Action,Thriller,Romance,Western,Children's,Mystery,Fantasy,...,Documentary,Comedy,Adventure,Sci-Fi,Horror,Crime,Musical,War,Animation,Drama
0,One Flew Over the Cuckoo's Nest (1975),Drama,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,James and the Giant Peach (1996),Animation|Children's|Musical,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,1,0,1,0
2,My Fair Lady (1964),Musical|Romance,2,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,Erin Brockovich (2000),Drama,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,"Bug's Life, A (1998)",Animation|Children's|Comedy,4,0,0,0,0,1,0,0,...,0,1,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3411,Lucie Aubrac (1997),Romance|War,3411,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3412,"Snows of Kilimanjaro, The (1952)",Adventure,3412,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3413,"Actor's Revenge, An (Yukinojo Henge) (1963)",Drama,3413,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3414,Cobra (1925),Drama,3414,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
users = pd.read_csv("../data/ml-1m/u_id_mapping_demographic.csv", sep="\t")
# users


users = users.sort_values(by="userID")

users = users.drop(columns=users.columns[0])
gender_map = {"M": 0, "F": 1}
users["Gender"] = users["Gender"].map(gender_map)
# user_features_numpy = users.to_numpy()
users

,Gender,Age,Occupation,ZipCode,userID
0,1,1,10,48067,0
1,0,56,16,70072,1
2,0,25,15,55117,2
3,0,45,7,02460,3
4,0,25,20,55455,4
...,...,...,...,...,...
6035,1,25,15,32603,6035
6036,1,45,1,76006,6036
6037,1,56,1,14706,6037
6038,1,45,0,01060,6038


In [18]:
import os
os.chdir("/Users/tahsinalamgirkheya/Desktop/work/reranking_fairnes")

from metrics.utils import map_age
users["Age_Code"] = users["Age"].apply(map_age)
users

,Gender,Age,Occupation,ZipCode,userID,Age_Code
0,1,1,10,48067,0,0
1,0,56,16,70072,1,6
2,0,25,15,55117,2,2
3,0,45,7,02460,3,4
4,0,25,20,55455,4,2
...,...,...,...,...,...,...
6035,1,25,15,32603,6035,2
6036,1,45,1,76006,6036,4
6037,1,56,1,14706,6037,6
6038,1,45,0,01060,6038,4


In [19]:
from mymetrics.GenrePrecisionMulti import GenrePrecisionMulti
from mymetrics.GenreNDCGMulti import GenreNDCGMulti
from mymetrics.GenreNDCG import GenreNDCG
####Import Metrics####
# from mymetrics.GenrePrecision import GenrePrecision

top_k = 20
###intialize them
gp = GenrePrecisionMulti(users, unique_genres, top_k)
gn = GenreNDCGMulti(users, unique_genres, top_k)
gn_old = GenreNDCG(users, unique_genres, top_k)


In [20]:
reco_matrix[0].shape
# new_rmatrix = np.
new_rmatrix = np.zeros(( reco_matrix.shape[0], 20), dtype=int)
for i in range(reco_matrix.shape[0]):
    new_rmatrix[i]=reco_matrix[i][:20]
new_rmatrix.shape


(6040, 20)

In [21]:
sens_attr = "Occupation"
gp_plain = gp.compute(new_rmatrix, movies, sens_attr)
gp_plain

(0.12864492710518602,
 array([0.01320081, 0.00859462, 0.00554757, 0.00145617, 0.00213506,
        0.00508368, 0.00108575, 0.00567179, 0.00317746, 0.01786337,
        0.00548233, 0.00889092, 0.00431231, 0.00616088, 0.00473964,
        0.00583507, 0.00603459, 0.02337292]))

In [22]:
gn_plain = gn.compute(new_rmatrix, movies, sens_attr)
gn_plain

(0.0476332850685533,
 array([0.00527466, 0.00287285, 0.0019415 , 0.00043556, 0.00078845,
        0.00159971, 0.00055686, 0.00158725, 0.00092582, 0.00711052,
        0.00286462, 0.00343642, 0.00167448, 0.0026343 , 0.00201081,
        0.00214203, 0.00219032, 0.00758712]))

In [23]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gp.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    # print(x)
    print(f"{x[0]} is the bias score for b={i}")


0.11610862078039716 is the bias score for b=1
0.10201391492336902 is the bias score for b=2
0.08195298832873467 is the bias score for b=3
0.05971713631311517 is the bias score for b=4
0.036305461219345865 is the bias score for b=5
0.024522942405449045 is the bias score for b=6
0.01728682352705113 is the bias score for b=7
0.014478615157006705 is the bias score for b=8


In [24]:
[gp_plain[0]]+xlist

[0.12864492710518602,
 0.11610862078039716,
 0.10201391492336902,
 0.08195298832873467,
 0.05971713631311517,
 0.036305461219345865,
 0.024522942405449045,
 0.01728682352705113,
 0.014478615157006705]

In [25]:
i=0
xlist=[]
for r in reco_matrix_fair:
    i=i+1
    x=gn.compute(np.array(r), movies, sens_attr)
    xlist.append(x[0])
    # print(x)
    print(f"{x[0]} is the bias score for b={i}")


0.042804248935800655 is the bias score for b=1
0.037519984214849283 is the bias score for b=2
0.0302512252480646 is the bias score for b=3
0.022384745314850487 is the bias score for b=4
0.014372592018730554 is the bias score for b=5
0.009805682072250988 is the bias score for b=6
0.006951188569421421 is the bias score for b=7
0.005450798057505372 is the bias score for b=8


In [26]:
[gn_plain[0]]+xlist

[0.0476332850685533,
 0.042804248935800655,
 0.037519984214849283,
 0.0302512252480646,
 0.022384745314850487,
 0.014372592018730554,
 0.009805682072250988,
 0.006951188569421421,
 0.005450798057505372]